In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jul 24 17:10:02 2020

@author: MonishRamadoss
"""
import glob
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras import Input, Model, models, layers
from medpy.io import load

tf.config.set_visible_devices([], 'GPU')

In [2]:
'''Confirming running on GPU'''

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
'''
CUDA_VISIBLE_DEVICES = 0,1
'''
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

'''config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' 
'''


"""gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)
    """
    
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 320140099517128528
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 2867950371251501899
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3048629862
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6843247979758528123
physical_device_desc: "device: 0, name: GeForce GTX 1650 with Max-Q Design, pci bus id: 0000:02:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 15712181477381356781
physical_device_desc: "device: XLA_GPU device"
]
Num GPUs Available:  1
1 Physical GPUs, 1 Logical GPU


In [3]:
"""mask_list = []
image_list = []
PATH = 'C:\\Users\\Andrew_Arbogast\\Desktop\\Codes\\UCAIDM\\CAIDM Project\\DataSets\\orCAScore Dataset\\Filtered\\Training_Set\\Train\\'
for file in glob.glob(PATH + '**\\*.mhd',  recursive=True):
if 'r.mhd' in file:
mask_list.append(file)
elif 'cti.mhd' in file:
image_list.append(file)

def im_gen():
data_set = list()
label_set = list()
for path0, path1 in zip(mask_list, image_list):
image, header_img = load(path1)
mask_img, header_mask = load(path0)
mask_img = np.transpose(image, (2, 0, 1))
image = np.transpose(image, (2,0,1))
for i in range(image.shape[0]):
data_set.append(np.expand_dims(image[i], 0))
label_set.append(np.expand_dims(mask_img[i], 0))
yield np.array(data_set), np.array(label_set)


def normalize(x):
x = np.asarray(x)
return (x - x.min()) / (np.ptp(x))

if __name__ == '__main__':

inputs = Input(shape=(1, 512, 512))

# --- Define model
# --- Define kwargs dictionary
kwargs = {
'kernel_size': 3,
'padding': 'same'}

# --- Define lambda functions
conv = lambda x, filters, strides : layers.Conv2D(filters=filters, strides=strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.LeakyReLU()(x)

# --- Define stride-1, stride-2 blocks
conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=2)))
# --- Define single transpose
tran = lambda x, filters, strides : layers.Conv2DTranspose(filters=filters, strides=strides, **kwargs)(x)
# --- Define transpose block
tran2 = lambda filters, x : relu(norm(tran(x, filters, strides=2)))
concat = lambda a, b : layers.Concatenate(axis=1)([a, b])


l1 = conv1(2, inputs)
#l2 = conv1(32, conv2(16, l1))
#l3 = conv1(48, conv2(32, l2))
#l4 = conv1(64, conv2(48, l3))
#l5 = conv1(128, conv2(64, l4))
#l6 = tran2(64, l5)
#l7 = tran2(48, conv1(64, concat(l4, l6)))
#l8 = tran2(32, conv1(48, concat(l3, l7)))
#l9 = tran2(16, conv1(32, concat(l2, l8)))
#l10 = conv1(16, l9)
#logits = layers.Conv2D(filters=1, **kwargs)(l10)
logits = layers.Conv2D(filters=1, **kwargs)(l1)
model = Model(inputs=inputs, outputs=logits)

train_examples , train_labels = im_gen()
train_labels = normalize(train_labels)
tf_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
tf_dataset = tf_dataset.shuffle(100).batch(1)
print(train_examples.shape)

optimizer=tf.keras.optimizers.Adam()
loss= tf.keras.losses.MeanSquaredError(reduction='sum_over_batch_size')
metrics = ['accuracy']
model.compile(optimizer = optimizer, loss= loss, metrics= metrics)
model.fit(train_examples, train_labels, batch_size=1, epochs = 10)


input("Pausing...") """

'mask_list = []\nimage_list = []\nPATH = \'C:\\Users\\Andrew_Arbogast\\Desktop\\Codes\\UCAIDM\\CAIDM Project\\DataSets\\orCAScore Dataset\\Filtered\\Training_Set\\Train\\\'\nfor file in glob.glob(PATH + \'**\\*.mhd\',  recursive=True):\nif \'r.mhd\' in file:\nmask_list.append(file)\nelif \'cti.mhd\' in file:\nimage_list.append(file)\n\ndef im_gen():\ndata_set = list()\nlabel_set = list()\nfor path0, path1 in zip(mask_list, image_list):\nimage, header_img = load(path1)\nmask_img, header_mask = load(path0)\nmask_img = np.transpose(image, (2, 0, 1))\nimage = np.transpose(image, (2,0,1))\nfor i in range(image.shape[0]):\ndata_set.append(np.expand_dims(image[i], 0))\nlabel_set.append(np.expand_dims(mask_img[i], 0))\nyield np.array(data_set), np.array(label_set)\n\n\ndef normalize(x):\nx = np.asarray(x)\nreturn (x - x.min()) / (np.ptp(x))\n\nif __name__ == \'__main__\':\n\ninputs = Input(shape=(1, 512, 512))\n\n# --- Define model\n# --- Define kwargs dictionary\nkwargs = {\n\'kernel_size\': 

In [4]:
mask_list = []
image_list = []
PATH = 'C:\\Users\\Andrew_Arbogast\\Desktop\\Codes\\UCAIDM\\CAIDM Project\\DataSets\\orCAScore Dataset\\Filtered\\Training_Set\\Train\\'
for file in glob.glob(PATH + '**\\*.mhd',  recursive=True):
    if 'r.mhd' in file:
        mask_list.append(file)
    elif 'cti.mhd' in file:
        image_list.append(file)

def im_gen():
    data_set = list()
    label_set = list()
    for path0, path1 in zip(mask_list, image_list):
        image, header_img = load(path1)
        mask_img, header_mask = load(path0)
        mask_img = np.transpose(image, (2, 0, 1))
        image = np.transpose(image, (2,0,1))
    for i in range(image.shape[0]):
        data_set.append(np.expand_dims(image[i], 0))
        label_set.append(np.expand_dims(mask_img[i], 0))
    return np.array(data_set), np.array(label_set)


def normalize(x):
    x = np.asarray(x)
    return (x - x.min()) / (np.ptp(x))

In [5]:
if __name__ == '__main__':

    inputs = Input(shape=(1, 512, 512))

    # --- Define model
    # --- Define kwargs dictionary
    kwargs = {
    'kernel_size': 3,
    'padding': 'same'}

    # --- Define lambda functions
    conv = lambda x, filters, strides : layers.Conv2D(filters=filters, strides=strides, **kwargs)(x)
    norm = lambda x : layers.BatchNormalization()(x)
    relu = lambda x : layers.LeakyReLU()(x)

    # --- Define stride-1, stride-2 blocks
    conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
    conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=2)))
    # --- Define single transpose
    tran = lambda x, filters, strides : layers.Conv2DTranspose(filters=filters, strides=strides, **kwargs)(x)
    # --- Define transpose block
    tran2 = lambda filters, x : relu(norm(tran(x, filters, strides=2)))
    concat = lambda a, b : layers.Concatenate(axis=1)([a, b])


    l1 = conv1(2, inputs)
    #l2 = conv1(32, conv2(16, l1))
    #l3 = conv1(48, conv2(32, l2))
    #l4 = conv1(64, conv2(48, l3))
    #l5 = conv1(128, conv2(64, l4))
    #l6 = tran2(64, l5)
    #l7 = tran2(48, conv1(64, concat(l4, l6)))
    #l8 = tran2(32, conv1(48, concat(l3, l7)))
    #l9 = tran2(16, conv1(32, concat(l2, l8)))
    #l10 = conv1(16, l9)
    #logits = layers.Conv2D(filters=1, **kwargs)(l10)
    logits = layers.Conv2D(filters=1, **kwargs)(l1)
    model = Model(inputs=inputs, outputs=logits)

    print("model creation completed, start creating images.")
   


model creation completed, start creating images.


In [7]:

train_examples, train_labels = im_gen()
train_labels = normalize(train_labels)
train_examples = normalize(train_examples)
tf_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
tf_dataset = tf_dataset.shuffle(100).batch(1)
print(train_examples.shape)

optimizer=tf.keras.optimizers.Adam()
loss= tf.keras.losses.MeanSquaredError(reduction='sum_over_batch_size')
metrics = ['accuracy']
model.compile(optimizer = optimizer, loss= loss, metrics= metrics)
#model.fit(train_examples, train_labels, batch_size=1, epochs = 10)
model.fit(train_examples, train_labels, batch_size=1, epochs=10)

(48, 1, 512, 512)
Epoch 1/10


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node functional_1/conv2d/Conv2D (defined at <ipython-input-7-d54a4523ab21>:13) ]] [Op:__inference_train_function_821]

Function call stack:
train_function
